<a href="https://colab.research.google.com/github/Tumars-web/E-commerce-user-growth-analysis/blob/main/notebooks/07_A_B_%E6%B5%8B%E8%AF%95%E8%AE%BE%E8%AE%A1%E4%B8%8E%E9%AA%8C%E8%AF%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import zipfile
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns


url = 'https://github.com/Tumars-web/E-commerce-user-growth-analysis/raw/main/cleaned_data.csv.zip'
zip_path = 'cleaned_data.zip'

# 下载压缩包
urllib.request.urlretrieve(url, zip_path)

# 解压缩
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# 读取 CSV
df = pd.read_csv('cleaned_data.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print("数据加载成功！")
df.head()

数据加载成功！


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# 你的数据加载代码
import zipfile
import urllib.request

url = 'https://github.com/Tumars-web/E-commerce-user-growth-analysis/raw/main/cleaned_data.csv.zip'
zip_path = 'cleaned_data.zip'

urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

df = pd.read_csv('cleaned_data.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print("数据加载成功！")

# ------------------------------------------------------
# 1. 给用户分组：模拟随机分配A/B组
user_ids = df['CustomerID'].unique()
np.random.seed(42)
group_map = pd.DataFrame({
    'CustomerID': user_ids,
    'group': np.random.choice(['A', 'B'], size=len(user_ids))
})

# 2. 计算每个用户的首次购买日期
first_order = df.groupby('CustomerID')['InvoiceDate'].min().reset_index()
first_order.rename(columns={'InvoiceDate': 'first_order_date'}, inplace=True)

# 3. 计算每个用户是否在首次购买后7天内复购
# 找到所有订单，跟首次购买日期对比
df = df.merge(first_order, on='CustomerID', how='left')
df['days_since_first_order'] = (df['InvoiceDate'] - df['first_order_date']).dt.days

# 复购定义：首次订单后的7天内有第二笔及以上订单，即days_since_first_order>0且<=7
repurchase_flag = df[(df['days_since_first_order'] > 0) & (df['days_since_first_order'] <= 7)].groupby('CustomerID').size()
repurchase_flag = repurchase_flag.reset_index()[['CustomerID']]
repurchase_flag['repurchase_7d'] = 1

# 合并复购标记，没有的用户默认为0
user_repurchase = pd.DataFrame({'CustomerID': user_ids})
user_repurchase = user_repurchase.merge(repurchase_flag, on='CustomerID', how='left').fillna(0)

# 4. 合并分组信息和复购标记
user_data = user_repurchase.merge(group_map, on='CustomerID', how='left')

# 5. 计算A/B组7天复购率
group_summary = user_data.groupby('group')['repurchase_7d'].agg(['mean', 'count', 'sum']).rename(columns={'mean':'repurchase_rate'})
print(group_summary)

# 6. 卡方检验判断差异显著性
contingency_table = pd.crosstab(user_data['group'], user_data['repurchase_7d'])
print("\n列联表：\n", contingency_table)

chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print(f"\n卡方检验结果：chi2={chi2:.4f}, p-value={p:.4f}")

if p < 0.05:
    print("结论：两组复购率有显著差异，实验组方案效果可能更好。")
else:
    print("结论：两组复购率无显著差异，无法确认方案效果有差别。")


数据加载成功！
       repurchase_rate  count    sum
group                               
A             0.058140   2150  125.0
B             0.053016   2188  116.0

列联表：
 repurchase_7d   0.0  1.0
group                   
A              2025  125
B              2072  116

卡方检验结果：chi2=0.4492, p-value=0.5027
结论：两组复购率无显著差异，无法确认方案效果有差别。


In [ ]:
#基于 RFM 分层的 A/B 测试分析，目标是分析“不同RFM层级的用户中，营销策略是否对7天复购率产生影响”

import pandas as pd
import numpy as np
from scipy import stats
import urllib.request
import zipfile

# ----------------- Step 1: 加载数据 -----------------
url = 'https://github.com/Tumars-web/E-commerce-user-growth-analysis/raw/main/cleaned_data.csv.zip'
zip_path = 'cleaned_data.zip'

urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

df = pd.read_csv('cleaned_data.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# ----------------- Step 2: 构建 RFM 层级 -----------------
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'UnitPrice': lambda x: (x * df.loc[x.index, 'Quantity']).sum()

}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# 分箱（越小越好 -> Recency 降序；越大越好 -> F/M 升序）
r_labels = range(4, 0, -1)
f_labels = m_labels = range(1, 5)

rfm['R'] = pd.qcut(rfm['Recency'], q=4, labels=r_labels)
rfm['F'] = pd.qcut(rfm['Frequency'], q=4, labels=f_labels)
rfm['M'] = pd.qcut(rfm['Monetary'], q=4, labels=m_labels)

rfm['RFM_Score'] = rfm[['R','F','M']].astype(str).sum(axis=1)

# 简化成层级标签
def segment(x):
    r, f, m = int(x['R']), int(x['F']), int(x['M'])
    if r >= 3 and f >= 3 and m >= 3:
        return 'Champion'
    elif r >= 3 and f >= 2:
        return 'Loyal'
    elif r >= 2 and f >= 1:
        return 'Potential'
    elif r >= 1:
        return 'At_Risk'
    else:
        return 'Hibernating'

rfm['RFM_Level'] = rfm.apply(segment, axis=1)

# ----------------- Step 3: 用户行为合并 -----------------
# 获取首次下单时间
first_order = df.groupby('CustomerID')['InvoiceDate'].min().reset_index()
first_order.rename(columns={'InvoiceDate': 'first_order_date'}, inplace=True)

df = df.merge(first_order, on='CustomerID', how='left')
df['days_since_first_order'] = (df['InvoiceDate'] - df['first_order_date']).dt.days

# 是否在7天内复购
repurchase_flag = df[(df['days_since_first_order'] > 0) & (df['days_since_first_order'] <= 7)].groupby('CustomerID').size()
repurchase_flag = repurchase_flag.reset_index()[['CustomerID']]
repurchase_flag['repurchase_7d'] = 1

# 所有用户+复购标记
user_repurchase = pd.DataFrame({'CustomerID': df['CustomerID'].unique()})
user_repurchase = user_repurchase.merge(repurchase_flag, on='CustomerID', how='left').fillna(0)

# 加上RFM层级
user_repurchase = user_repurchase.merge(rfm[['CustomerID', 'RFM_Level']], on='CustomerID', how='left')

# ----------------- Step 4: 每层RFM做随机A/B组 + 复购率分析 -----------------
results = []

np.random.seed(42)
for level in user_repurchase['RFM_Level'].dropna().unique():
    group_data = user_repurchase[user_repurchase['RFM_Level'] == level].copy()

    # A/B分组
    group_data['group'] = np.random.choice(['A', 'B'], size=len(group_data))

    # 分组复购率
    summary = group_data.groupby('group')['repurchase_7d'].agg(['mean', 'count', 'sum']).rename(columns={'mean':'repurchase_rate'})
    summary['RFM_Level'] = level
    summary.reset_index(inplace=True)

    # 卡方检验
    cont = pd.crosstab(group_data['group'], group_data['repurchase_7d'])
    if cont.shape == (2,2):  # 确保两组都有数据
        chi2, p, _, _ = stats.chi2_contingency(cont)
    else:
        p = np.nan  # 无法做检验

    summary['p_value'] = p
    results.append(summary)

# ----------------- Step 5: 输出结果 -----------------
final_results = pd.concat(results).sort_values(by=['RFM_Level', 'group'])
print(final_results[['RFM_Level', 'group', 'repurchase_rate', 'count', 'sum', 'p_value']])


   RFM_Level group  repurchase_rate  count   sum   p_value
0    At_Risk     A         0.027523    545  15.0  0.570732
1    At_Risk     B         0.020408    539  11.0  0.570732
0   Champion     A         0.107623    669  72.0  0.086113
1   Champion     B         0.140575    626  88.0  0.086113
0      Loyal     A         0.028169    284   8.0  0.435741
1      Loyal     B         0.015625    320   5.0  0.435741
0  Potential     A         0.021472    652  14.0  0.073254
1  Potential     B         0.039829    703  28.0  0.073254


In [ ]:
# 实战A/B测试设计与分析：对 Champion 和 Potential 用户做精准营销策略测试
import numpy as np
from scipy import stats

# 只分析 Champion 和 Potential 用户
target_levels = ['Champion', 'Potential']
target_users = user_repurchase[user_repurchase['RFM_Level'].isin(target_levels)].copy()

results = []

np.random.seed(42)
for level in target_levels:
    group_data = target_users[target_users['RFM_Level'] == level].copy()

    # 分组模拟：A = 控制组，B = 实验组
    group_data['group'] = np.random.choice(['A', 'B'], size=len(group_data))

    # 复购率统计
    summary = group_data.groupby('group')['repurchase_7d'].agg(['mean', 'count', 'sum']).rename(columns={'mean':'repurchase_rate'})
    summary['RFM_Level'] = level
    summary.reset_index(inplace=True)

    # 卡方检验
    cont = pd.crosstab(group_data['group'], group_data['repurchase_7d'])
    if cont.shape == (2,2):
        chi2, p, _, _ = stats.chi2_contingency(cont)
    else:
        p = np.nan

    summary['p_value'] = p
    results.append(summary)

# 合并输出
ab_results = pd.concat(results).sort_values(by=['RFM_Level', 'group'])
print(ab_results[['RFM_Level', 'group', 'repurchase_rate', 'count', 'sum', 'p_value']])


   RFM_Level group  repurchase_rate  count   sum   p_value
0   Champion     A         0.124424    651  81.0  0.990895
1   Champion     B         0.122671    644  79.0  0.990895
0  Potential     A         0.026087    690  18.0  0.365263
1  Potential     B         0.036090    665  24.0  0.365263
